In [1]:
import numpy as np
import pandas as pd

C:\Users\Windows\AppData\Local\Temp\ipykernel_21060\1662815981.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train_buf = pd.read_csv(train_url) #training set

test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test_buf = pd.read_csv(test_url) #test set


In [3]:
train = train_buf
test = test_buf

In [4]:
## task 8
print(train["Age"].median())

train["Age"] = train["Age"].fillna(train["Age"].median())
test["Age"] = test["Age"].fillna(test["Age"].median())

28.0


In [5]:
## task 9
print(train["Embarked"].mode())

train.loc[train["Embarked"] == "S", "Embarked"] = 0
train.loc[train["Embarked"] == "C", "Embarked"] = 1
train.loc[train["Embarked"] == "Q", "Embarked"] = 2

train.loc[train["Sex"] == "male", "Sex"] = 0
train.loc[train["Sex"] == "female", "Sex"] = 1

test.loc[test["Embarked"] == "S", "Embarked"] = 0
test.loc[test["Embarked"] == "C", "Embarked"] = 1
test.loc[test["Embarked"] == "Q", "Embarked"] = 2

test.loc[test["Sex"] == "male", "Sex"] = 0
test.loc[test["Sex"] == "female", "Sex"] = 1

0    S
Name: Embarked, dtype: object


In [6]:
print(train["Pclass"].unique())

[3 1 2]


In [7]:
train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])
train["Sex"] = train["Sex"].fillna(train["Sex"].mode()[0])
print(train.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


C:\Users\Windows\AppData\Local\Temp\ipykernel_21060\3363175144.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train["Embarked"] = train["Embarked"].fillna(train["Embarked"].mode()[0])
C:\Users\Windows\AppData\Local\Temp\ipykernel_21060\3363175144.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train["Sex"] = train["Sex"].fillna(train["Sex"].mode()[0])


In [19]:
# task 12
# task 13
def prepare(dataframe):
    # feature = ["Sex","Age"]
    feature = ["Pclass","Sex","Age","Embarked"]
    x = np.array(dataframe[feature].values, dtype=np.double)

    # x = np.insert(x, -1, [x[:,0]*x[:,0], x[:,1]*x[:,1], np.abs(x[:,2]-15), np.abs(x[:,2]-20), np.abs(x[:,2]-25), np.abs(x[:,2]-30), np.abs(x[:,2]-35), np.abs(x[:,2]-40)], axis=1)
    # x = np.insert(x, -1, [x[:,0]*x[:,0], x[:,1]*x[:,1], x[:,2]*x[:,2], x[:,3]*x[:,3], x[:,0]*x[:,1], x[:,0]*x[:,2], x[:,0]*x[:,3], x[:,1]*x[:,2], x[:,1]*x[:,3], x[:,2]*x[:,3]], axis=1)

    x_mi = np.min(x, axis=0)
    x_ma = np.max(x, axis=0)
    

    if "Survived" not in dataframe.columns:
        return x
    y = np.array(dataframe["Survived"].values, dtype=np.double)
    return x, y, x_mi, x_ma

In [9]:
## task 10
x,y,x_mi,x_ma = prepare(train)
x = (x - x_mi) / (x_ma - x_mi)

z = np.array([0.5]*x.shape[1], dtype=float)
print(x.shape, y.shape)

(891, 14) (891,)


In [10]:
def H(x):
    return 1 / (1 + np.exp(-np.dot(x,z)))

r_max = 0.003
r_min = 0.00001
iter = 20000

for iii in range(1,iter+1):
    r = r_max - (r_max - r_min)*iii/iter
    sum = np.zeros(x.shape[1])
    for j in range(len(y)):
        sum += (y[j] - H(x[j]))*x[j]
    z += r*sum
    if(iii % 1000==0):
        print("iterations : ",iii)
        print("    zeta : ",z)


iterations :  1000
    zeta :  [-0.35404295  2.55327427 -0.86344954 -0.41347336  2.55327427  0.40114732
 -1.78730352 -4.22882176 -3.00543467  0.43011768  1.7377223   1.39996635
 -0.62616612  1.53753834]
iterations :  2000
    zeta :  [-0.5071823   2.68104653 -0.92852903 -0.08778292  2.68104653  0.94043367
 -2.29257489 -4.6457088  -3.61097119  0.4908172   1.99327846  1.53293161
 -0.98044596  2.00126953]
iterations :  3000
    zeta :  [-0.6469791   2.70067102 -0.99167254  0.10671727  2.70067102  1.18208665
 -2.44822059 -4.74115425 -3.78743302  0.47079706  2.10461619  1.57303153
 -1.19282629  2.2153651 ]
iterations :  4000
    zeta :  [-0.74636765  2.7104655  -1.03592236  0.22480117  2.7104655   1.29572977
 -2.49933743 -4.7814329  -3.83242787  0.43108042  2.14508315  1.59082708
 -1.32863212  2.33740433]
iterations :  5000
    zeta :  [-0.81631988  2.71924946 -1.06755348  0.30061887  2.71924946  1.35421072
 -2.51576917 -4.80634983 -3.83333835  0.3927026   2.15652329  1.60082575
 -1.4193606

In [11]:
x_test, y_test,pp,pp = prepare(train)
x_test = (x_test - x_mi) / (x_ma - x_mi)
y_pred = np.zeros(len(y_test))
for i in range(len(y_test)):
    y_pred[i] = 0 if H(x_test[i]) < 0.5 else 1

print("accuracy of train: ",np.sum(y_pred == y_test)/len(y_test))

accuracy of train:  0.813692480359147


In [12]:
x_test = prepare(test)
x_test = (x_test - x_mi) / (x_ma - x_mi)
y_pred = np.zeros(len(x_test))
for i in range(len(x_test)):
    y_pred[i] = 0 if H(x_test[i]) < 0.5 else 1

pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": y_pred.astype(int)}).to_csv("submission.csv", index=False)

In [20]:
# OT3
# OT4

# Gradient Descent
x,y,x_mi,x_ma = prepare(train)
x = (x - x_mi) / (x_ma - x_mi)

z = np.array([0.5]*x.shape[1], dtype=np.double)

r_max = 0.003
r_min = 0.0001
iter = 2000

for iii in range(1,iter+1):
    r = r_max - (r_max - r_min)*iii/iter
    sum = np.zeros(x.shape[1])
    for j in range(len(y)):
        sum += (y[j] - np.dot(z, x[j]))*x[j]
    z += r*sum

MSE = np.sum((y - x@z)**2)/len(y)
print("zeta : ",z)
print("MSE : ",MSE)

zeta :  [-0.09088149  0.60886717  0.44454662  0.13934095]
MSE :  0.16946707481047424


In [21]:
# Pseudo Inverse
x,y,x_mi,x_ma = prepare(train)
x = (x - x_mi) / (x_ma - x_mi)

z_inv = np.linalg.inv(x.T @ x) @ x.T @ y
MSE = np.sum((y - x @ z_inv)**2)/len(y)
print("zeta : ",z_inv)
print("MSE : ",MSE)

zeta :  [-0.09088149  0.60886717  0.44454662  0.13934095]
MSE :  0.16946707481047424


In [22]:
for i in np.unique(np.insert(x[:,0:2],-1,x[:,3],axis=1), axis=0):
    print(i)

[0. 0. 0.]
[0. 0. 1.]
[0.  0.5 0. ]
[0.  0.5 1. ]
[0. 1. 0.]
[0. 1. 1.]
[0.5 0.  0. ]
[0.5 0.  1. ]
[0.5 0.5 0. ]
[0.5 0.5 1. ]
[0.5 1.  0. ]
[0.5 1.  1. ]
[1. 0. 0.]
[1. 0. 1.]
[1.  0.5 0. ]
[1.  0.5 1. ]
[1. 1. 0.]
[1. 1. 1.]
